In [2]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from tqdm import tqdm

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on the GPU")
else:
    device = torch.device("cpu")
    print("running on the CPU")

running on the CPU


In [4]:
training_data = np.load("training_data.npy", allow_pickle=True)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(300, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.softmax(x, dim=1)

In [6]:
tr_size = round(len(training_data)*0.8)
tr_set = training_data[:tr_size]
te_set = training_data[tr_size:]

In [7]:
net = Net().to(device)
print(net)

Net(
  (fc1): Linear(in_features=300, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=2, bias=True)
)


In [10]:
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(0, len(te_set), BATCH_SIZE)):
            batch_X = [x for x, y_ in te_set[i:i+BATCH_SIZE]]
            batch_y = [y_ for x, y_ in te_set[i:i+BATCH_SIZE]]

            tensor_batch_X = torch.tensor(batch_X).float().view(-1, 300).to(device)
            tensor_batch_y = torch.tensor(batch_y).float().to(device)
            batch_out = net(tensor_batch_X)

            out_maxes = [torch.argmax(val) for val in batch_out]
            target_maxes = [torch.argmax(val) for val in tensor_batch_y]
            
            for i, j in zip(out_maxes, target_maxes):
                if i == j:
                    correct +=1
                total += 1
    return round(correct/total, 3)

In [21]:
BATCH_SIZE = 100
def train(net):
    EPOCHS = 15
    loss_function = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    loss = 1000000000
    test_accuracy_at_each_epoch = []
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(tr_set), BATCH_SIZE)):
            batch_X = [x for x, y_ in tr_set[i:i+BATCH_SIZE]]
            batch_y = [y_ for x, y_ in tr_set[i:i+BATCH_SIZE]]

            tensor_batch_X = torch.tensor(batch_X).float()
            tensor_batch_y = torch.tensor(batch_y).float()

            tensor_batch_X, tensor_batch_y = tensor_batch_X.to(device), tensor_batch_y.to(device)

            net.zero_grad()

            output = net(tensor_batch_X.view(-1, 300))
            loss = loss_function(output, tensor_batch_y)
            loss.backward()
            optimizer.step()
      
        accuracy = test(net)
        test_accuracy_at_each_epoch.append(accuracy)
        print(f"\nLoss: {loss}")
        print(f"\nAccuracy: {accuracy}")
    return test_accuracy_at_each_epoch

In [ ]:
test_accuracy_epoch = train(net)

 11%|████                                 | 1985/18164 [00:18<02:35, 104.28it/s]

In [ ]:
print(test_accuracy_epoch)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(list(range(1,30*8+1)), test_accuracy_epoch)
plt.show()

In [ ]:
torch.save(net.state_dict(), "nn_dict")
torch.save(net, "nn_object")